# This notebook will take your regression or classification problem, the y variables (dependent) and x variables (independent) and determine which have what level of contribution to your dataset.
## This will allow you to visualize the information context of Xn to Y, and decide what to keep in future analyses, such as development of lienar regression

In [ ]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

In [ ]:
analysis = 'ols_linear_regression/regression'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/final_analyses/interaction_analysis/spcorrel_interaction_effect/ferguson_memory_derivatives_with_clinical_data.csv'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/vta_published_t_connectivity'
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(conn_path.split('.')[0], f'{analysis}')
    #out_dir = r'path to out dir here'
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    
    print('I have set pathnames in the Mac style')
    print('I will save to :', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\AD_to_memory_net'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

## Import Pre-prepared Regression Data

## Prepare for regression

In [ ]:
variables_to_change = {}
variables_to_change['original'] = [
    '% Change from baseline (ADAS-Cog11)'
  
    ]
variables_to_change['destination'] = [
    'percent_change_adascog11'
    
    ]
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(conn_path)
try:
    change_string = ''
    for i in range (0, len(variables_to_change['original'])):
        change_string += f"variables_to_change['original'][{i}]: variables_to_change['destination'][{i}],"
    change_string = 'data_df.rename(columns={' + change_string[0:-1] + '})'
    data_df = eval(change_string)
except:
    print('no change necessary')

data_df = data_df.reset_index(drop=True)

#Select specific subgroup
# outlier_index = (data_df['percent_change_adascog11'] <= -50)
# data_df = data_df.loc[outlier_index, :]

# Remove outlier
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

try:    
    data_df.pop('Unnamed: 0')
    data_df.pop('Patient # CDR, ADAS')
    # data_df.pop('Age at DOS')
    data_df.pop('Randomization Arm')
    data_df.pop('Baseline ADAS-Cog11')
    data_df.pop('Baseline CDR (sum of squares)')
    data_df.pop('% Change from baseline (CDR)')
    data_df.dropna(inplace=True)
    data_df.reset_index(drop=True, inplace=True)
except:
    print('none to pop')
    
#Standardize the data
preserved_df = data_df.copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
# data_df = data_df.apply(scaler.fit_transform(), axis=0)
data_df[data_df.columns] = scaler.fit_transform(data_df[data_df.columns])


# #move outcomes to start
try:
    first_column = data_df.pop('percent_change_adascog11')  # Remove column and store it in a variable
    data_df.insert(0, 'percent_change_adascog11', first_column)  # Insert column at the first position
except:
    print('Could not move outcomes to start')

display(data_df)

In [ ]:
#Check multicollinearity
mc_test = data_df.copy()
mc_test.pop('percent_change_adascog11')

from calvin_utils.statistical_measurements import calculate_vif
calculate_vif(mc_test)

## Generate Statsmodel Linear Model

In [ ]:
#function attempt
from calvin_utils.custom_regressions import basic_trivariate_regression, generalizable_multivariate_regression
# results = basic_trivariate_regression(x=col2, y=col3, z=ycolumn_of_interest, conjoined_df=merged_dataframe)
# data_df.pop('RMap')

#Select all regressors
all_indeps = data_df.columns.tolist()[-1:]
#Select specific regressors
# all_indeps = np.array(all_indeps)[[-1]]
# all_indeps = ['pca_component_0']#, 'Ventral_Attention', 'Visual', 'Dorsal_Attention', 'Limbic', 'Default']#, 'SBC'] #,Thal_Ant_Med_L 'PFC_Mid_Vent'] #,'', #'Front_Sup_L'] #, 'SBC'] #, 'Cereb_IX_R', 'Occ_med_L', 'Par_Lat_L', 'SBC_L', 'Sbc_Retrospl_L', 'Temp_Lat_R']

#Select outcome variable 
dep_var = 'percent_change_adascog11'

# if isinstance(all_indeps[0], tuple):
#     print('Found tuple, correcting')
#     new_indeps = []
#     for indep in all_indeps:
#         new_indeps.append(indep[0])
#     data_df = data_df.set_axis([['percent_change_adascog11'] + new_indeps], axis=1, inplace=False)
#     all_indeps = data_df.columns.tolist()[1:]
#     results = generalizable_multivariate_regression(data_df, 'percent_change_adascog11', all_indeps)
# else:
results, formula = generalizable_multivariate_regression(data_df, f'{dep_var}', all_indeps)
print(results.summary2())
print(results.mse_model, results.mse_total, results.mse_resid)

In [ ]:
results.rsquared_adj

In [ ]:
save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.makedirs(out_dir)
    try:
        with open(os.path.join(out_dir, f'summary_{formula}.txt'), 'w') as f:
            f.write(results.summary().as_text())
    except:
        with open(os.path.join(out_dir, f'summary_full_formula.txt'), 'w') as f:
            f.write(results.summary().as_text())
        print('saved as full formula')

In [ ]:
#Get Pearson R for the model
correlation_selections = ['pearsonr', 'spearmanr']
correlation = 'pearsonr'

##--------------------------------USER INPUT--------------------------------
from scipy.stats import pearsonr, spearmanr
#receive predictions from model
predictions = results.predict(data_df.dropna(inplace=True))

#Perform correlation
if correlation == 'spearmanr':
    r, p = spearmanr(data_df['percent_change_adascog11'], predictions)
elif correlation == 'pearsonr':
    r, p = pearsonr(data_df['percent_change_adascog11'], predictions)
else:
    print('Could not perform correlation, please retry')
    
#Put predictions in DF
pearson_df = pd.DataFrame({'R': r, 'P': p}, index=[0])
display(pearson_df)

#Save predicitons
save=False
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
    try:
        pearson_df.to_csv(out_dir + f'{correlation}_results{formula}.csv')
    except:
        pearson_df.to_csv(out_dir + f'{correlation}_results_full_formula.csv')


In [ ]:
#Get residual diagnostics
from calvin_utils.statistical_measurements import model_diagnostics
fig = model_diagnostics(results)
fig.savefig(os.path.join(out_dir, f'multivariate_regression.png'))
fig.savefig(os.path.join(out_dir, f'multivariate_regression.svg'))

In [ ]:
#Visualize
#Partial Regression Grid
from statsmodels.graphics.regressionplots import plot_partregress_grid
sns.set_style('white')
sns.set_palette('Greys', 1, desat=1)
fig = plt.figure(figsize=(40, 80))
plot_partregress_grid(results, fig=fig)

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)

    try:
        fig.savefig(os.path.join(out_dir, f'partial_regression_grid{formula}.png'))
        fig.savefig(os.path.join(out_dir, f'partial_regression_grid{formula}.svg'))
    except:
        fig.savefig(os.path.join(out_dir, f'partial_regression_grid_full_formula.png'))

In [ ]:
#Single Coefficient Partial Regression Plot
from statsmodels.graphics.regressionplots import plot_partregress
##--Exog_others set to all co-regressors to generate plot when all held stable--##
# fig = plot_partregress(endog='percent_change_adascog11', exog_i='SBC',
#                         exog_others=all_indeps[0:6],
#                         data=data_df, obs_labels=False)

#---Change exog_others to set the independent variables to account for
# fig = plot_partregress(endog='percent_change_adascog11', exog_i='memory_roi',
#                         exog_others=['conjunction_rios_r_to_ferguson_t_95th_percentile_stdevthresh_0_voxthresh_10_absval_True_cluster_6'],
#                         data=data_df, obs_labels=False)

# plt.xlabel('Residual')
# plt.ylabel('Percent Change ADAS-Cog 11')
# plt.title('')

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    fig.savefig(os.path.join(out_dir, 'memory_roi.png'))

In [ ]:
from statsmodels.graphics.regressionplots import plot_ccpr
sns.set_style('white')
# sns.set_palette('blue')

# ax = plot_ccpr(results, 'SBC')
# plt.xlabel('Independent Variable by Order')
# plt.ylabel('Error (Residual + BX)')
# plt.title('')

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    # ax.savefig(os.path.join(out_dir, f'ccpr_plot_{formula}.png'))


In [ ]:
##-----------Generate Residual Plot---------------------------------------------
#Make Predictions
sns.set_style('white')
plt.figure(figsize=(10,10))
colors = ['#696969', '#BAB1B0', '#D3D3D3', '#D8D6D5']
customPalette = sns.set_palette(sns.color_palette(colors))
predictions = results.fittedvalues
r, p = pearson_df['R'][0], pearson_df['P'][0]

# plt.axes(aspect='equal')
sns.residplot(x=preserved_df.iloc[:,0], y=results.resid, lowess=True)
plt.title(f'r = {r:.2f}, p = {p:.5f}')
plt.xlabel('Actual % Improvement ADAS-Cog 11')
plt.ylabel('Model Residual (Standard Deviations)')

if save:
    try:
        plt.savefig(os.path.join(out_dir, f'residplot_{formula}.png'))
    except:
        plt.savefig(os.path.join(out_dir, f'summary_full_formula.png'))

## Generate a New Dataframe from Specified Columns

In [ ]:
new_df = data_df.copy()
# columns_to_extract = ['percent_change_adascog11', 'Ventral_Attention', 'Frontoparietal']
new_df = new_df.iloc[:, 0:]
new_df.head(2)


## Perform K Folds with statsmodels

In [ ]:
from calvin_utils.cross_validation import k_folds

#Write the function to be passed to the LOO analysis
k=5
try:
    data_df.dropna(inplace=True)
    # data_df.reset_index(inplace=True, drop=True)
except:
    print('Cannot operate on DF')
function = 'generalizable_multivariate_regression(train_df, "percent_change_adascog11", train_df.columns.tolist()[1:])'

results_df = k_folds(data_df=new_df,train_function=function, k=k)
display(results_df)

# results_df.to_csv(os.path.join(out_dir, f'{k}_folds_analysis_{formula}.csv'))

## Perform leave-one-out analysis with statsmodels

In [ ]:
from calvin_utils.cross_validation import leave_one_out
#Write the function to be passed to the LOO analysis
function = 'generalizable_multivariate_regression(iterating_df, "percent_change_adascog11", iterating_df.columns.tolist()[1:])'

#Pass the variables to the LOO analysis, with specific outcomes to extract as a list. 
loo_results = leave_one_out(data_df=new_df, outcome_var='percent_change_adascog11', function=function, leave_out_variable='index')
display(loo_results)


In [ ]:
# loo_df = pd.DataFrame(loo_results)
from scipy.stats import spearmanr, pearsonr

#Calculate pearson predicted vs actual metrics
loo_results.dropna(inplace=True)
loo_results['loocv_pearson_r'], loo_results['loocv_pearson_p'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
loo_results['loocv_r_squared'] = np.square(loo_results['loocv_pearson_r'])
_, loo_results['loocv_pearson_p_greater'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
_, loo_results['loocv_pearson_p_lesser'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='less')
_, loo_results['loocv_pearson_p_two-sided'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='two-sided')


#Calculate MSE predicted vs actual metrics
loo_results['loocv_mse'] = (np.sum(np.square((loo_results['actuals']-loo_results['predictions'])))) / len(loo_results['actuals'])
loo_results['loocv_rmse'] = np.sqrt(loo_results['loocv_mse'])

#Calculate Mean Absolute Error
loo_results['mean_absolute_error'] = np.sum(np.absolute(loo_results['predictions']-loo_results['actuals']))/len(loo_results['predictions'])

# x = data_df['percent_change_adascog11'].values.tolist()
# display(loo_results['predictions'][1][0])
display(loo_results)

In [ ]:
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    try:
        loo_results.to_csv(os.path.join(out_dir, f'ols_leave_one_out_{formula}.csv'))
    except:
        loo_results.to_csv(os.path.join(out_dir, f'ols_leave_one_out_full_formula.csv'))


print('Saved to: ' + out_dir)

## Perform Permutation Analysis of the Regression

In [ ]:
from calvin_utils.import_matrices import import_matrices_from_folder

#This code is expecting a dataframe with patients in rows, outcomes in a column, and voxels in all other columns. 
#The voxel column titles at this point must be strings

# Load the flattened dataframe
# df = import_matrices_from_folder(clin_path, file_pattern='/*.nii')
df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/response_topology/voxelwise_glm/age_interaction_rios_vtas/t_values_topology/matrix_to_test.csv')
df

In [ ]:
# Achieve Voxels to Regress Upon 
import numpy as np
import pandas as pd
# Load the csv with voxel coordinates in it
coordinate_df = pd.read_csv(r'/Users/cu135/Dropbox (Partners HealthCare)/memory/final_analyses/maxima_and_minima_rois/ferguson_maxima_coordinates.csv')

#Convert coordinates to flattened coordinates
import nibabel as nib
from nimlab import datasets as nimds
from calvin_utils.matrix_utilities import convert_coordinate_to_index, convert_voxel_index_to_flattened_index

#Achieve relevant brain mask
mni_mask = nimds.get_img("mni_icbm152")
mask_affine = mni_mask.affine
mask_shape = mni_mask.shape

# Convert MNI coordinates to voxel space coordinates (i, j, k)
coordinate_df['voxel_coords'] = coordinate_df.apply(lambda row: convert_coordinate_to_index((row['x'], row['y'], row['z']), mask_affine), axis=1)

# Convert voxel space coordinates (i, j, k) to index coordinates
coordinate_df['index_coords'] = coordinate_df['voxel_coords'].apply(lambda coords: convert_voxel_index_to_flattened_index(coords, mask_shape))

# Display the resulting DataFrame
coordinate_df

In [ ]:

def find_closest_values(list_one, list_two):
    list_one_int = [int(x) for x in list_one]
    list_two_int = [int(x) for x in list_two]

    intersection_list = list(set(list_one_int) & set(list_two_int))
    complement_list = list(set(list_one_int) - set(intersection_list))

    closest_values = []
    for value in complement_list:
        min_distance = None
        closest_value = None
        for item in list_two_int:
            distance = abs(value - item)
            if min_distance is None or distance < min_distance:
                min_distance = distance
                closest_value = item
        closest_values.append(closest_value)

    return closest_values

In [ ]:
# Now, double check that the coordiantes you are going to test are in the flattened niftis
nifti_coordinates = df.columns.values.tolist()
test_coordinates = coordinate_df.loc[:,'index_coords'].values.tolist()
test_coordinates = [str(test_coordinate) for test_coordinate in test_coordinates]
shared_elements = list(set(nifti_coordinates).intersection(test_coordinates))
print('Number of shared coordinates: ', np.count_nonzero(shared_elements))
print('Numbder of total coordiantes: ', len(coordinate_df['index_coords'].values))
print('---')
print(shared_elements)
print('Now attempting to find the missing value')
missing_vals = find_closest_values(test_coordinates, nifti_coordinates[3:])[0]
print('Missing value(s): ', missing_vals)
test_coordinates.append(str(missing_vals))
test_coordinates
shared_elements = list(set(nifti_coordinates).intersection(test_coordinates))
print('Number of shared coordinates: ', np.count_nonzero(shared_elements))
print('Numbder of total coordiantes: ', len(coordinate_df['index_coords'].values))

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

X = df.loc[:, shared_elements]
X = sm.add_constant(X)
y = df.loc[:, 'stdev_improvement_adascog11']
model = sm.OLS(y, X).fit()
observed_r_squared = model.rsquared
print(model.summary2())

In [ ]:
# Perform PALM assessment of the Regression
from calvin_utils.palm import permute_row
from tqdm import tqdm
n_permutations = 10000
# permuted_indices = permute_row(df.columns.values[3:])
empiric_r_squareds = []
y = df.loc[:, 'stdev_improvement_adascog11']
for i in tqdm(range(0, n_permutations)):
    randomly_sampled_indices = np.random.choice(df.columns.values[3:], size=len(shared_elements), replace=False)
    X = df.loc[:, randomly_sampled_indices]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    empiric_r_squareds.append(model.rsquared)
print(empiric_r_squareds)
print('empiric p: ', np.count_nonzero(empiric_r_squareds>observed_r_squared)/n_permutations)

## Perform partial regression

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
def partial_regression_plots(data_df, dep_var, all_indeps):
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    # Define the formula for the full model
    formula = f"{dep_var} ~ {' + '.join(all_indeps)}"

    # Fit the full model
    results = smf.ols(formula, data=data_df).fit()

    # Create the partial regression plots
    plot_data = pd.melt(data_df, id_vars=[dep_var], value_vars=all_indeps, var_name='indep_var')
    grid = sns.FacetGrid(plot_data, col='indep_var', col_wrap=4, sharex=False, sharey=True)
    grid.map(sns.regplot, 'value', dep_var, truncate=False)
    grid.set_axis_labels('Regressed Connectivity (Standard Deviation)', 'Improvement (Standard Deviation)')
    grid.set_titles(col_template="{col_name}")

    for ax, indep_var in zip(grid.axes.flat, all_indeps):
        # Calculate Spearman correlation and p-value

        # Get partial regression coefficients
        pr_formula = f"{dep_var} ~ {indep_var} + {' + '.join([var for var in all_indeps if var != indep_var])}"
        pr_results = smf.ols(pr_formula, data=data_df).fit()
        coef = pr_results.params[indep_var]
        pval = pr_results.pvalues[indep_var]
        print(coef, pval)

        #Get the Rho of the residuals to outcomes (currently unused)
        r, p = spearmanr(pr_results.resid, data_df[dep_var])


        # Annotate the plot with correlation and p-value
        ax.annotate(f"coeff = {coef:.2f}, p = {pval:.5f}", xy=(.5, 1.0), xycoords='axes fraction',
                    xytext=(0, 0), textcoords='offset points',
                    ha='center', va='top',
                    bbox=dict(boxstyle='round,pad=0.0', alpha=0.0),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        return grid

grid = partial_regression_plots(data_df=data_df, dep_var=dep_var, all_indeps=all_indeps)

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    try:
        grid.savefig(os.path.join(out_dir, f'partial_regression_grid{formula}.png'))
        grid.savefig(os.path.join(out_dir, f'partial_regression_grid{formula}.svg'))
    except:
        grid.savefig(os.path.join(out_dir, f'partial_regression_grid_full_formula.png'))

## Some Simple Neural Network Stuff Below


Calculate the residuals for each regression, less one independent variable

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

def create_residual_df(data_df, dep_var, all_indeps):
    # Initialize empty dictionary to store residuals
    residual_dict = {}

    # Loop through each independent variable
    for indep_var in all_indeps:
        # Define the formula for the partial model
        formula = f"{dep_var} ~ {' + '.join([var for var in all_indeps if var != indep_var])}"

        # Fit the partial model
        results = smf.ols(formula, data=data_df).fit()

        # Add the residuals to the dictionary with the independent variable name as the key
        residual_dict[indep_var] = results.resid

    # Add the dependent variable to the dictionary
    residual_dict[dep_var] = data_df[dep_var]

    # Convert the dictionary to a dataframe
    residual_df = pd.DataFrame(residual_dict)

    return residual_df
residual_df = create_residual_df(data_df, dep_var, all_indeps)
display(residual_df)

Perform linear regressions using the residual against the dependent variable

In [ ]:
import seaborn as sns
from scipy.stats import spearmanr

def plot_residuals(residual_df, dep_var, indep_vars):
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    # Melt the residual dataframe
    melted_df = pd.melt(residual_df, id_vars=[dep_var], value_vars=indep_vars, var_name='indep_var')

    # Create the facet grid of lmplots
    facet_plot = sns.lmplot(y=dep_var, x='value', data=melted_df,
                            col='indep_var', col_wrap=4,
                            truncate=False, sharex=False, sharey=True)
    facet_plot.set_titles(col_template="{col_name}")
    facet_plot.set_axis_labels('Regressed Connectivity (Standard Deviation)', 'Improvement (Standard Deviation)')
    
    # Calculate Spearman correlation and p-value for each independent variable
    correlations = {}
    for iv in indep_vars:
        r, p = spearmanr(residual_df[iv], residual_df[dep_var])
        correlations[iv] = (r, p)

    # Add annotations to the facet grid
    for i, ax in enumerate(facet_plot.axes.flat):
        iv = indep_vars[i]
        r, p = correlations[iv]
        ax.annotate(f"r = {r:.2f}, p = {p:.5f}", xy=(.5, 1.0), xycoords='axes fraction',
                    xytext=(0, 0), textcoords='offset points',
                    ha='center', va='top',
                    bbox=dict(boxstyle='round,pad=0.0', alpha=0.0),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

    return facet_plot

plot_residuals(residual_df, dep_var, all_indeps)

## Get the results of the partial regression from a multivariate regression

In [ ]:
import statsmodels.api as sm

def multiple_regression(residual_df, dep_var):
    X = residual_df.drop([dep_var], axis=1)
    y = residual_df[dep_var]

    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    return model

model = multiple_regression(residual_df, dep_var)
print(model.summary2())

In [ ]:
import re

def create_residual_df(data_df, dep_var, all_indeps):
    # Initialize empty dictionary to store residuals
    residual_dict = {}

    # Loop through each independent variable
    for indep_var in all_indeps:
        # Define the formula for the partial model
        partial_formula = f"{dep_var} ~ {' + '.join([var for var in all_indeps if var != indep_var])}"

        # Fit the partial model
        partial_results = smf.ols(partial_formula, data=data_df).fit()

        # Add the residuals to the dictionary with the independent variable name as the key
        partial_col_name = re.sub(r'\W+', '', indep_var) + '_partial_dep'
        residual_dict[partial_col_name] = partial_results.resid

        # Define the formula for the second model
        second_formula = f"{indep_var} ~ {' + '.join([var for var in all_indeps if var != indep_var])}"

        # Fit the second model
        second_results = smf.ols(second_formula, data=data_df).fit()

        # Add the residuals to the dictionary with the independent variable name as the key
        second_col_name = re.sub(r'\W+', '', indep_var) + '_partial_indep'
        residual_dict[second_col_name] = second_results.resid

    # Add the dependent variable to the dictionary
    dep_col_name = re.sub(r'\W+', '', dep_var)
    residual_dict[dep_col_name] = data_df[dep_var]

    # Convert the dictionary to a dataframe
    residual_df = pd.DataFrame(residual_dict)
    residual_df = residual_df.drop(columns=[dep_col_name]) # remove dependent variable column
    return residual_df

res_df = create_residual_df(residual_df, dep_var, all_indeps)


In [ ]:
display(res_df)

In [ ]:
import seaborn as sns

# Get the unique first words in the column names
first_words = set([col.split()[0].split('_')[0] for col in res_df.columns])
print(first_words)

col_pairs = []
for word in first_words:
    cols = [col for col in res_df.columns if word in col]
    if len(cols) == 2:
        col_pairs.append(cols)
print(col_pairs)
# Create facet grid of lmplots for columns with the same first word
g = sns.FacetGrid(res_df, col=col_pairs, col_wrap=3)
g.map(sns.lmplot, x=col_pairs[i][0], y=col_pairs[i][1], data=res_df, scatter_kws={'s': 10}) 


In [ ]:
sns.lmplot(x="Visual_partial_indep", y="Visual_partial_dep", data=res_df)


In [ ]:
import seaborn as sns

# Get the unique first words in the column names
first_words = set([col.split()[0].split('_')[0] for col in res_df.columns])
print(first_words)

col_pairs = []
for word in first_words:
    cols = [col for col in res_df.columns if word in col]
    if len(cols) == 2:
        col_pairs.append(cols)

# Create facet grid of lmplots for columns with the same first word
g = sns.FacetGrid(res_df, col=first_words, col_wrap=3)

# Map lmplot onto each pair of columns
for pair in col_pairs:
    x, y = pair
    g.map(sns.lmplot, x=x, y=y, data=res_df, scatter_kws={'s': 10})
    
# Show the plot
plt.show()
